# `mammos_entity.io`: reading and writing entities

In [1]:
import mammos_entity as me
import mammos_units as u

## Supported file format

`mammos_entity.io` can read and write csv files containing entity like objects (entities, quantities, or other array-like data) in tabluar format. For entities and quantities information about ontology and units are included as additional metadata. Details of the file format are explained in the [`mammos-entity.io` api reference](https://mammos-project.github.io/mammos/api/mammos_entity.io.html)

## MaMMoS CSV

We create some artificial data to write to a csv file.

In [2]:
Ms = me.Ms([600, 650, 700], "kA/m")
T = me.T([1, 2, 3])
theta_angle = [0, 0.5, 0.7] * u.rad
demag_factor = me.Entity("DemagnetizingFactor", [1 / 3, 1 / 3, 1 / 3])
comments = ["Some comment", "Some other comment", "A third comment"]

### Writing
We can write them to a csv file as shown in the following cell. Names of the keyword arguments determine column names in the file.

In [3]:
me.io.entities_to_csv("example.csv", Ms=Ms, T=T, angle=theta_angle, demag_factor=demag_factor, comment=comments)

This has produced the following file:

In [4]:
print(open("example.csv").read())  # noqa: SIM115

#mammos csv v1
#SpontaneousMagnetization,ThermodynamicTemperature,,DemagnetizingFactor,
#https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25,https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f,,https://w3id.org/emmo/domain/magnetic_material#EMMO_0f2b5cc9-d00a-5030-8448-99ba6b7dfd1e,
#kA / m,K,rad,,
Ms,T,angle,demag_factor,comment
600.0,1.0,0.0,0.3333333333333333,Some comment
650.0,2.0,0.5,0.3333333333333333,Some other comment
700.0,3.0,0.7,0.3333333333333333,A third comment



### Reading
We can read it back in and get a container object (called EntityCollection) containing all columns:

In [5]:
content = me.io.entities_from_csv("example.csv")
content

EntityCollection(
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([600., 650., 700.]), unit='kA / m'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K'),
    angle=<Quantity [0. , 0.5, 0.7] rad>,
    demag_factor=Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333])),
    comment=array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object),
)

The recommended way of accessing the data is by using the individual elements. This preserves the correct data type:

In [6]:
content.Ms

Entity(ontology_label='SpontaneousMagnetization', value=array([600., 650., 700.]), unit='kA / m')

In [7]:
content.T

Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K')

In [8]:
content.angle

<Quantity [0. , 0.5, 0.7] rad>

In [9]:
content.demag_factor

Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333]))

In [10]:
content.comment

array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object)

We can also get a `pandas` dataframe of the data we have read. This is designed as a convenience functions but due to limitation of `pandas` we loose ontology information. This is why we recommend using the individual elements directly where possible. The columns names consist of short name and units (where columns have a unit):

In [11]:
content.to_dataframe()

,Ms (kA / m),T (K),angle (rad),demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


We can also get a dataframe without units in the column names:

In [12]:
content.to_dataframe(include_units=False)

,Ms,T,angle,demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


### Reading with `pandas`
If we only need the numerical data but not the entity information, we can also read the csv file with pandas:

In [13]:
import pandas as pd

pd.read_csv("example.csv", comment="#")

,Ms,T,angle,demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


### Check that data has not changed
We can compare with the original data:

In [14]:
Ms == content.Ms

True

In [15]:
T == content.T

True

In [16]:
theta_angle == content.angle

array([ True,  True,  True])

In [17]:
demag_factor == content.demag_factor

True

In [18]:
comments == content.comment

array([ True,  True,  True])